Here is a simple example of how the package can be used and what requirements must be met in order to be able to use the full range of functions.
A basic requirement is the availability of downloaded satellite image data (so far only Sentinel-2 is supported). The data structure should look similar to this: 

![Example data structure](./img/example_data_structure.png)

The most important thing is that the bands in the folder are marked with a "B" and the number and stored as GeoTIFF, e.g. ..._B02.tif, ..._B12.tif. If the data is to be pre-processed for a deep learning model, then a file with the corresponding labels must also be provided for each image and contain the name annotation (see example: annotation.tif).

In [1]:
# Make sure the package is installed
# pip install satellite_datacube

In [1]:
from satellite_datacube.datacube import SatelliteDataCube 

# Define folder of satellite data and specify some parameters (read more in the doc-string)
path_of_SITS = r"D:\SatelliteDataCube\Chimanimani" # path_of_SITS = ".../Sentinel2"
satellites = ["S1", "S2"]
dcParameter = { "timeseriesLength": 6 , "badPixelLimit": 15 , "patchSize": 128}

In [3]:
S2_datacube = SatelliteDataCube(base_folder=path_of_SITS, parameters=dcParameter, load_data=False, preprocess=False)

-------------------- Chimanimani --------------------
Initializing data-cube with following parameter:
- base folder: D:\SatelliteDataCube\Chimanimani
- length of timeseries: 6
- limit of bad pixel per satellite image in timeseries: 15%
- patch size of 128


In [3]:
# Preprocessing data:
# 1. Building global data 
#    - selecting images for timeseries with length parameter 
#    - building/loading one mask for entire timeseries 
# 2. Create patches of selected timeseries for image and mask data
#    - split images/masks in smaller patches with proceeded size 
#    - stack patches together with shape pattern: NxTxCxHxW 
#       - img = multi-spectral images for each timestep
#       - msk = single binary masks for each timestep
#       - msk_gb = single binary masks for whole timeseries (missing the time axis)
S2_datacube.preprocess()

Building global mask of datacube
Selecting timeseries of length 6 with bad pixel limit of 15 % for each satellite image


KeyboardInterrupt: 